# Yet Another OpenPose 

---
## Environment

In [1]:
#verify user
!whoami
#verify user account, if running on google cloud, otherwise ignore
!gcloud auth list
#which Environment/virtualenv running in
import sys
import os

print("Python running from:",sys.prefix)
print("Current working dir",os.getcwd())

michael_zl_prime
                  Credentialed Accounts
ACTIVE  ACCOUNT
        312164605303-compute@developer.gserviceaccount.com
*       michael.zl.prime@gmail.com

To set the active account, run:
    $ gcloud config set account `ACCOUNT`

Python running from: /usr
Current working dir /home/michael_zl_prime/project_drive/project


In [2]:
#start tensor board
# must run 
#/usr/local/bin/tensorboard serve --logdir gs://dl_training_results/tensorboard --port 8889 --bind_all

In [3]:
#start TPU
!gcloud compute tpus start node-1 --zone us-central1-c
#stop TPU
#!gcloud compute tpus stop node-1 --zone us-central1-c

ERROR: (gcloud.compute.tpus.start) INVALID_ARGUMENT: cannot start an already running instance


---
## Config

In [4]:
import default_config as cfg

#import local_storage_config as storage_cfg
import remote_storage_config as storage_cfg
cfg.__dict__.update(storage_cfg.__dict__)

import tpu_training.TPU_config as TPU_config
cfg.__dict__.update(TPU_config.__dict__) #comment out to disable TPU training
cfg.RUN_NAME="Full_model_2_2augs_dp01_15joint_nofinalbn-" #for reference 

In [5]:
import tensorflow as tf
print("Tensorflow version:",tf.version.VERSION)

Tensorflow version: 2.1.0-dev20191203


In [6]:
if cfg.TPU_MODE:
    print("Testing results bucket connectivity")
    !touch /tmp/test
    !gsutil cp /tmp/test {cfg.TENSORBOARD_PATH}/test
    !gsutil rm {cfg.TENSORBOARD_PATH}/test
    !gsutil cp /tmp/test {cfg.CHECKPOINTS_PATH}/test
    !gsutil rm {cfg.CHECKPOINTS_PATH}/test
    print("Testing dataset bucket connectivity")
    !gsutil ls gs://{cfg.GCS_TFRECORDS_BUCKETNAME} | head -4
    print("Testing TPU connectivity")
    !nmap -Pn -p8470 {cfg.TPU_IP}
    import tpu_training.init_TPU as init_TPU
    strategy,resolver=init_TPU.init_tpu() #This must be run before any imports!!!!

Testing results bucket connectivity
Copying file:///tmp/test [Content-Type=application/octet-stream]...
/ [1 files][    0.0 B/    0.0 B]                                                
Operation completed over 1 objects.                                              
Removing gs://dl_training_results/tensorboard/test...
/ [1 objects]                                                                   
Operation completed over 1 objects.                                              
Copying file:///tmp/test [Content-Type=application/octet-stream]...
/ [1 files][    0.0 B/    0.0 B]                                                
Operation completed over 1 objects.                                              
Removing gs://dl_training_results/checkpoints/test...
/ [1 objects]                                                                   
Operation completed over 1 objects.                                              
Testing dataset bucket connectivity
gs://datasets_bucket_a/training-

INFO:tensorflow:Initializing the TPU system: grpc://10.0.3.2:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


In [7]:
import datetime
import dataset_functions
import models.six_stage_linear_model as model
import callbacks
import dataset_builder
import load_weights
import loss_metrics
from utils import now

In [8]:
# import importlib as il
# il.reload() #useful to reload any changes

---
## Dataset

In [9]:
model_ds=model.ModelDatasetComponent(cfg)

tfrecord_files_train=dataset_builder.get_tfrecord_filenames(cfg.TRAIN_TFRECORDS,cfg)
tfrecord_files_valid=dataset_builder.get_tfrecord_filenames(cfg.VALID_TFRECORDS,cfg)
print("Found the following training TFrecords:\n","\n".join(tfrecord_files_train))
print("Found the following validation TFrecords:\n","\n".join(tfrecord_files_valid))

print("Building training dataset")
dst=dataset_builder.build_training_ds(tfrecord_files_train,model_ds.place_training_labels,cfg)
print("Training dataset shape:",dst)
print("Building validation dataset")
dsv=dataset_builder.build_validation_ds(tfrecord_files_valid,model_ds.place_training_labels,cfg)
print("Validation dataset shape:",dsv)
STEPS_PER_EPOCH = int(cfg.DATASET_SIZE / cfg.BATCH_SIZE)

Retrieving TFrecords from: gs://datasets_bucket_a/training
Retrieving TFrecords from: gs://datasets_bucket_a/validation
Found the following training TFrecords:
 gs://datasets_bucket_a/training-001.tfrecords
gs://datasets_bucket_a/training-002.tfrecords
gs://datasets_bucket_a/training-003.tfrecords
gs://datasets_bucket_a/training-004.tfrecords
gs://datasets_bucket_a/training-005.tfrecords
gs://datasets_bucket_a/training-006.tfrecords
gs://datasets_bucket_a/training-007.tfrecords
gs://datasets_bucket_a/training-008.tfrecords
gs://datasets_bucket_a/training-009.tfrecords
gs://datasets_bucket_a/training-010.tfrecords
gs://datasets_bucket_a/training-011.tfrecords
gs://datasets_bucket_a/training-012.tfrecords
gs://datasets_bucket_a/training-013.tfrecords
gs://datasets_bucket_a/training-014.tfrecords
gs://datasets_bucket_a/training-015.tfrecords
gs://datasets_bucket_a/training-016.tfrecords
gs://datasets_bucket_a/training-017.tfrecords
gs://datasets_bucket_a/training-018.tfrecords
gs://datase

In [10]:
#Test element
# dst_iter=iter(dst)
# sample_elem=next(dst_iter)
# print("Dataset shape:",dst) #this should match the model input, and output stages

In [11]:
#testing mask
# sample_elem=next(dst_iter)
# m=sample_elem[1][1][0,...,0]
# import matplotlib.pyplot as plt
# plt.imshow(m)
# plt.colorbar()

---
## Model
### Load weights

In [ ]:
if cfg.ASK_FOR_CHECKPOINTS:
    checkpoint,starting_epoch=load_weights.checkpoints_prompt(cfg)
else:
    checkpoint=None
    starting_epoch=0

Found these checkpoints:
0.Dont load checkpoint
1 .gs://dl_training_results/checkpoints/Full_model_2_2augs_dp0111Wed1219-1833/-E0009.ckpt_temp_1d8239e45732435f97e3b43074192752/part-00000-of-00002
2 .gs://dl_training_results/checkpoints/Full_model_2_2augs_dp0111Wed1219-1833/-E0009.ckpt_temp_1d8239e45732435f97e3b43074192752/part-00001-of-00002
3 .gs://dl_training_results/checkpoints/Full_model_2_2augs_dp0111Wed1219-1842/-E0001.ckpt
4 .gs://dl_training_results/checkpoints/Full_model_2_2augs_dp0111Wed1219-1842/-E0002.ckpt
5 .gs://dl_training_results/checkpoints/Full_model_2_2augs_dp0111Wed1219-1842/-E0003.ckpt
6 .gs://dl_training_results/checkpoints/Full_model_2_2augs_dp01_15joint11Wed1219-1917/-E0001.ckpt
7 .gs://dl_training_results/checkpoints/Full_model_2_2augs_dp01_15joint11Wed1219-1917/-E0002.ckpt
8 .gs://dl_training_results/checkpoints/Full_model_2_2augs_dp01_15joint11Wed1219-1917/-E0003.ckpt
9 .gs://dl_training_results/checkpoints/Full_model_2_2augs_dp01_15joint11Wed1219-1917/-E0004

### Build Model

In [ ]:
model_maker=model.ModelMaker(cfg) #must be outside scope to keep the graph clean
tf.keras.backend.clear_session() #to clean to backaend from the imported model
def define():
    train_model,test_model=model_maker.create_models()
    
#     if cfg.INCLUDE_MASK:
#         losses=[loss_metrics.MaskedMeanAbsoluteError()
#                 ,loss_metrics.MaskedMeanAbsoluteError()
#                 ,loss_metrics.MaskedMeanAbsoluteError()
#                 ,loss_metrics.MaskedMeanAbsoluteError()
#                 ,loss_metrics.MaskedMeanSquaredError()
#                 ,loss_metrics.MaskedMeanSquaredError()]
                      
#     else:
#         raise NotImplementedError       
    
    #this must match the model output order
#     metrics=[
#           []
#          ,[]
#          ,[]
#          ,[]
#          ,[tf.keras.metrics.MeanAbsoluteError()]    
#          ,[tf.keras.metrics.MeanAbsoluteError()]
#         ]
    
    train_model.compile(
                    optimizer=tf.keras.optimizers.Adam(cfg.BASE_LEARNING_RATE)                   
                    ,loss=loss_metrics.MaskedMeanSquaredError()
                    ,loss_weights=[10,10,10,10,0.2,0.2]
                    #,metrics=metrics                           
                   )
    return train_model,test_model

if cfg.TPU_MODE:
    with strategy.scope():
        train_model,test_model=define()
        if (checkpoint):
            train_model.load_weights(checkpoint)
else:
    train_model,test_model=define()
    if (checkpoint):
        train_model.load_weights(checkpoint)

### Callbacks

In [ ]:
all_callbacks=[
    callbacks.make_checkpoint_callback(cfg)
    ,callbacks.make_tensorboard_callback(cfg)
    ,callbacks.make_LRscheduler_callback(cfg.LEARNING_RATE_SCHEDUELE)
    ,callbacks.PrintLR()
    ,tf.keras.callbacks.TerminateOnNaN()
]


In [ ]:
#run to clean tensorboard results
#!gsutil -m rm -r {cfg.TENSORBOARD_PATH}/*
#!gsutil -m rm -r {cfg.CHECKPOINTS_PATH}/*

---
## Training
Actually training

In [ ]:
train_history=train_model.fit(
    dst
    ,epochs=cfg.TRAINING_EPOCHS
    ,steps_per_epoch=STEPS_PER_EPOCH
    ,validation_data=dsv
    ,callbacks=all_callbacks
    ,initial_epoch=starting_epoch
)

---
### Save results

In [ ]:
tmp_path='gs://dl_training_results/tmp/tensorflow/temp_weights'
train_model.save_weights(tmp_path)

cpu_train_model,cpu_test_model=define()

cpu_train_model.load_weights(tmp_path)
cpu_test_model.load_weights(tmp_path)

cpu_test_model.save(cfg.MODELS_PATH+"/test_"+now()+cfg.RUN_NAME) 
cpu_train_model.save(cfg.MODELS_PATH+"/train_"+now()+cfg.RUN_NAME) 

---
### Shutdown

In [ ]:
#shut down TPU
!gcloud compute tpus stop node-1 --zone us-central1-c

In [ ]:
!gcloud compute instances stop instance-1 --zone us-central1-c